In [14]:
# General imports
import numpy as np

# Qiskit ansatz circuits
from qiskit.circuit.library import EfficientSU2, TwoLocal

# Qiskit primitives
from qiskit.primitives import Estimator as QiskitEstimator
from qiskit.primitives import Sampler as QiskitSampler

# Qiskit runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session

# Qiskit optimization
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.workflows import QUBO_transformer
from qiskit_optimization.passes.qubo_to_ising import QUBO2Ising
from qiskit_optimization.passes.eval_solution import EvaluateProgramSolution
from qiskit_optimization.passes.qubo_unroller import UnrollQUBOVariables

# Docplex
from docplex.mp.model import Model

# SPSA
from qiskit_optimization.spsa import minimize_spsa

## Load the Runtime (if using)

In [15]:
#service = QiskitRuntimeService()

In [16]:
#backend = service.get_backend('ibm_nazca')

## Build optimization problem using docplex
Define problem using standard classical representation. **This is the classical input.  Nothing quantum is here.**

In [30]:
from rustworkx import PyDiGraph
# 0 is NYC
# 1 is Tokyo
# 2 is Seoul
# 3 is London
# 4 is paris


A = np.array([[0, 14+10/60, 15+40/60, 6+40/60, 7+35/60],
          [12+50/60, 0, 2+30/60, 14+25/60, 14+35/60],
          [14+15/60, 2+20/60, 0, 14+25/60, 13+35/60],
          [7+40/60, 13+35/60, 12+20/60, 0, 1+15/60],
          [8+30/60, 13+15/60, 12+15/60, 1+5/60, 0],
         ])

In [46]:
 mdl = Model(name="TSP")
n = 5
x = {(i, k): mdl.binary_var(name=f"x_{i}_{k}") for i in range(n) for k in range(n)}
tsp_func = mdl.sum(
    A[i, j] * x[(i, k)] * x[(j, (k + 1) % n)]
    for i in range(n)
    for j in range(n)
    for k in range(n)
    if i != j
)
mdl.minimize(tsp_func)
for i in range(n):
    mdl.add_constraint(mdl.sum(x[(i, k)] for k in range(n)) == 1)
for k in range(n):
    mdl.add_constraint(mdl.sum(x[(i, k)] for i in range(n)) == 1)

print(mdl.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: TSP

Minimize
 obj: [ 28.333333333333 x_0_0*x_1_1 + 25.666666666667 x_0_0*x_1_4
      + 31.333333333333 x_0_0*x_2_1 + 28.500000000000 x_0_0*x_2_4
      + 13.333333333333 x_0_0*x_3_1 + 15.333333333333 x_0_0*x_3_4
      + 15.166666666667 x_0_0*x_4_1 + 17 x_0_0*x_4_4
      + 25.666666666667 x_0_1*x_1_0 + 28.333333333333 x_0_1*x_1_2
      + 28.500000000000 x_0_1*x_2_0 + 31.333333333333 x_0_1*x_2_2
      + 15.333333333333 x_0_1*x_3_0 + 13.333333333333 x_0_1*x_3_2
      + 17 x_0_1*x_4_0 + 15.166666666667 x_0_1*x_4_2
      + 25.666666666667 x_0_2*x_1_1 + 28.333333333333 x_0_2*x_1_3
      + 28.500000000000 x_0_2*x_2_1 + 31.333333333333 x_0_2*x_2_3
      + 15.333333333333 x_0_2*x_3_1 + 13.333333333333 x_0_2*x_3_3
      + 17 x_0_2*x_4_1 + 15.166666666667 x_0_2*x_4_3
      + 25.666666666667 x_0_3*x_1_2 + 28.333333333333 x_0_3*x_1_4
      + 28.500000000000 x_0_3*x_2_2 + 31.333333333333 x_0_3*x_2_4
      + 15.33333333333

## Convert to our `QuadraticProgram` format

In [47]:
qp = QuadraticProgram.from_docplex_mp(mdl)

## Perform transformations to QUBO and return new program
**Nothing quantum happens here either**

In [48]:
qubo_transformer = QUBO_transformer()
qubo = qubo_transformer.run(qp)

## All the quantum stuff is here ⚛️
### Convert from QUBO to Ising

In [49]:
hamiltonian = QUBO2Ising().run(qubo)
hamiltonian.num_qubits

25

### Standard cost function definition

In [50]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    cost = estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    return cost

### Setup estimator and sampler instances

In [51]:
#session = Session(backend=backend)
#estimator = Estimator(session=session, options={"shots": int(1e4)})
#sampler = Sampler(session=session, options={"shots": int(1e4)})
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit_aer.primitives import Estimator as AerEstimator

estimator = AerEstimator()
sampler = AerSampler()

### Perform minimization

In [52]:
ansatz = TwoLocal(hamiltonian.num_qubits, 'ry', 'cx', 'linear', reps=2)
x0 = 2*np.pi*np.random.random(size=ansatz.num_parameters)
res = minimize_spsa(cost_func, x0, args=(ansatz, hamiltonian, estimator), maxiter=150)

### Computute distribution at found minimum

In [53]:
# Assign solution parameters to ansatz
qc = ansatz.assign_parameters(res.x)
qc.measure_all()
samp_dist = sampler.run(qc, shots=int(1e4)).result().quasi_dists[0]
# Close the session since we are now done with it
#session.close()

### Express quantum solution as QUBO variable values
Convert bit-string solutions to variable values in QUBO problem

In [54]:
res = EvaluateProgramSolution(qubo).run(samp_dist)
res

(2077.833333333335,
 array([0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 0]))

## Classical postprocess
Unrolling QUBO variables back to the original problem definition. **This is classical processing. Nothing quantum happens here**

In [55]:
solution = UnrollQUBOVariables(qubo_transformer).run(res)
solution

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0.])

## Intepretation of solution
Depends on input, but is completely separate from quantum having done the processing for the solution.

In [56]:
x = solution
n = int(np.sqrt(len(x)))
route = []  # type: List[Union[int, List[int]]]
for p__ in range(n):
    p_step = []
    for i in range(n):
        if x[i * n + p__]:
            p_step.append(i)
    if len(p_step) == 1:
        route.extend(p_step)
    else:
        route.append(p_step)
route

[2, [3, 4], 0, [], 1]

In [43]:
from itertools import permutations

def brute_force_tsp(w, N):
    a = list(permutations(range(1, N)))
    last_best_distance = np.inf
    for i in a:
        distance = 0
        pre_j = 0
        for j in i:
            distance = distance + w[j, pre_j]
            pre_j = j
        distance = distance + w[pre_j, 0]
        order = (0,) + i
        if distance < last_best_distance:
            best_order = order
            last_best_distance = distance
            print("order = " + str(order) + " Distance = " + str(distance))
    return last_best_distance, best_order

In [57]:
brute_force_tsp(A.T, 5)

order = (0, 1, 2, 3, 4) Distance = 39.916666666666664
order = (0, 1, 2, 4, 3) Distance = 38.0
order = (0, 3, 4, 2, 1) Distance = 36.666666666666664


(36.666666666666664, (0, 3, 4, 2, 1))